This file is used to explore the dataset and test the pipeline, need to be removed in the final version.

In [1]:
import kagglehub
import os
from datasets import load_dataset
import datasets
import os
path = "/Users/fege/Desktop/project/MultimodalToolBox/data/amazon-reviews-data-2023/data"

/Users/fege/Desktop/project/MultimodalToolBox/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_All_Beauty", trust_remote_code=True, cache_dir=path)

Generating full split: 100%|██████████| 112590/112590 [00:06<00:00, 17697.92 examples/s]


In [5]:
dataset['full'][:3]

{'main_category': ['All Beauty', 'All Beauty', 'All Beauty'],
 'title': ['Howard LC0008 Leather Conditioner, 8-Ounce (4-Pack)',
  'Yes to Tomatoes Detoxifying Charcoal Cleanser (Pack of 2) with Charcoal Powder, Tomato Fruit Extract, and Gingko Biloba Leaf Extract, 5 fl. oz.',
  'Eye Patch Black Adult with Tie Band (6 Per Pack)'],
 'average_rating': [4.8, 4.5, 4.4],
 'rating_number': [10, 3, 26],
 'features': [[], [], []],
 'description': [[], [], []],
 'price': ['None', 'None', 'None'],
 'images': [{'hi_res': [None,
    'https://m.media-amazon.com/images/I/71i77AuI9xL._SL1500_.jpg'],
   'large': ['https://m.media-amazon.com/images/I/41qfjSfqNyL.jpg',
    'https://m.media-amazon.com/images/I/41w2yznfuZL.jpg'],
   'thumb': ['https://m.media-amazon.com/images/I/41qfjSfqNyL._SS40_.jpg',
    'https://m.media-amazon.com/images/I/41w2yznfuZL._SS40_.jpg'],
   'variant': ['MAIN', 'PT01']},
  {'hi_res': ['https://m.media-amazon.com/images/I/71g1lP0pMbL._SL1500_.jpg',
    'https://m.media-amazon.

In [ ]:
dataset = load_dataset()

In [7]:
dataset['full'].save_to_disk("/Users/fege/Desktop/project/MultimodalToolBox/data/amazon-back")

Saving the dataset (1/1 shards): 100%|██████████| 112590/112590 [00:00<00:00, 920618.31 examples/s]


In [88]:
ds = datasets.load_from_disk("/Users/fege/Desktop/project/MultimodalToolBox/data/amazon-reviews-data-2023/data_origin")

In [152]:
ds[0]

{'main_category': 'All Beauty',
 'title': 'Howard LC0008 Leather Conditioner, 8-Ounce (4-Pack)',
 'average_rating': 4.8,
 'rating_number': 10,
 'features': [],
 'description': [],
 'price': 'None',
 'images': {'hi_res': [None,
   'https://m.media-amazon.com/images/I/71i77AuI9xL._SL1500_.jpg'],
  'large': ['https://m.media-amazon.com/images/I/41qfjSfqNyL.jpg',
   'https://m.media-amazon.com/images/I/41w2yznfuZL.jpg'],
  'thumb': ['https://m.media-amazon.com/images/I/41qfjSfqNyL._SS40_.jpg',
   'https://m.media-amazon.com/images/I/41w2yznfuZL._SS40_.jpg'],
  'variant': ['MAIN', 'PT01']},
 'videos': {'title': [], 'url': [], 'user_id': []},
 'store': 'Howard Products',
 'categories': [],
 'details': '{"Package Dimensions": "7.1 x 5.5 x 3 inches; 2.38 Pounds", "UPC": "617390882781"}',
 'parent_asin': 'B01CUPMQZE',
 'bought_together': None,
 'subtitle': None,
 'author': None}

In [147]:
dt = ds.select([1])
def get_unique_item_id(dataset):
    unique_ids = list(set(ds['parent_asin']))
    id_mapping = {asin: idx for idx, asin in enumerate(unique_ids)}
    with open('/Users/fege/Desktop/project/MultimodalToolBox/data/amazon-reviews-data-2023/data/parent_asin_mapping.txt', 'w') as f:
        for asin, idx in id_mapping.items():
            f.write(f"{asin} {idx}\n")
    return id_mapping
id_mapping = get_unique_item_id(dt)

In [148]:
def transform_dataset(dataset):

    def transform_item(item):
        item_id = id_mapping[item['parent_asin']]
        r_id = []
        t = []

        if item['title']:
            r_id.append('title')
            t.append(item['title'])

        if item['description'] and item['description'] not in ['None', '']:
            r_id.extend(['description'] * len(item['description']))
            t.extend(item['description'])

        if item['images']['large'] and item['images']['large'] not in ['None', '']:
            r_id.extend(['has_image'] * len(item['images']['large']))
            t.extend([e.split('/')[-1] for e in item['images']['large']])

        return {
            'item_id': [item_id] * len(r_id),
            'r_id': r_id,
            't': t
        }

    new_ds = {'item_id': [], 'r_id': [], 't': []}
    for idx in range(dataset.num_rows):
        t = transform_item(dataset[idx])
        new_ds['item_id'].extend(t['item_id'])
        new_ds['r_id'].extend(t['r_id'])
        new_ds['t'].extend(t['t'])
    new_ds = datasets.Dataset.from_dict(new_ds)
    return new_ds


In [149]:
new_ds = transform_dataset(dt)

In [150]:
new_ds.save_to_disk('/Users/fege/Desktop/project/MultimodalToolBox/data/amazon-reviews-data-2023/data')

Saving the dataset (1/1 shards): 100%|██████████| 3/3 [00:00<00:00, 388.55 examples/s]


In [151]:
new_ds

Dataset({
    features: ['item_id', 'r_id', 't'],
    num_rows: 3
})